In [ ]:
import time
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import datetime

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role

In [ ]:
with_categories = False

base_job_name = 'deepar-biketrain-with-dynamic-feat'

In [ ]:
bucket = 'mehrshad-ml-sagemaker'
prefix = 'deepar/bikerental'


s3_data_path = "{}/{}/data_dynamic".format(bucket, prefix)
s3_output_path = "{}/{}/output".format(bucket, prefix)

In [ ]:
s3_data_path,s3_output_path

In [ ]:
def write_to_s3(filename, bucket, key):
    with open(filename,'rb') as f: # Read in binary mode
        return boto3.Session().resource('s3').Bucket(bucket).Object(key).upload_fileobj(f)

In [ ]:
# Upload one or more training files and test files to S3
write_to_s3('train_dynamic_feat.json',bucket,'deepar/bikerental/data_dynamic/train/train_dynamic_feat.json')
write_to_s3('test_dynamic_feat.json',bucket,'deepar/bikerental/data_dynamic/test/test_dynamic_feat.json')

In [ ]:
sagemaker_session = sagemaker.Session()
role = get_execution_role()

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri
image_name = get_image_uri(boto3.Session().region_name, 'forecasting-deepar')

In [ ]:
image_name

In [ ]:
freq='H' 
prediction_length = 288 

context_length = 288

In [ ]:
estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sagemaker_session,
    image_name=image_name,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.c5.4xlarge',
    base_job_name=base_job_name,
    output_path="s3://" + s3_output_path
)

In [ ]:
freq, context_length, prediction_length

In [ ]:
hyperparameters = {
    "time_freq": freq,
    "epochs": "400",
    "early_stopping_patience": "40",
    "mini_batch_size": "64",
    "learning_rate": "5E-4",
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
    "cardinality" : "auto" if with_categories else ''
}

In [ ]:
hyperparameters

In [ ]:
estimator.set_hyperparameters(**hyperparameters)

In [ ]:
# referring to train path and test path
data_channels = {
    "train": "s3://{}/train/".format(s3_data_path),
    "test": "s3://{}/test/".format(s3_data_path)
}

In [ ]:
data_channels

In [ ]:
estimator.fit(inputs=data_channels)

In [ ]:
job_name = estimator.latest_training_job.name

In [ ]:
job_name = 'deepar-biketrain-with-dynamic-feat-2018-12-17-17-54-02-175'

In [ ]:
print ('job name: {0}'.format(job_name))

In [ ]:
# Create an endpoint for real-time predictions
endpoint_name = sagemaker_session.endpoint_from_job(
    job_name=job_name,
    initial_instance_count=1,
    instance_type='ml.m4.xlarge',
    deployment_image=image_name,
    role=role
)

In [ ]:
print ('endpoint name: {0}'.format(endpoint_name))